In [6]:
import numpy as np
from numpy.random import normal
from numpy.random import uniform
from numpy.linalg import inv
from math import sqrt

In [28]:
N, p, sigma, low, high = 10, 3, 1, -1, 1
b = np.array([[2, 5, -4, 0]]).T

In [29]:
sigma_squared_hat_biased = []
sigma_squared_hat_unbiased = []
for i in range(10000):
    X = uniform(low, high, size=(N, p+1))
    X[:,0] = 1
    y = X.dot(b) + normal(0, sigma, (N, 1))
    b_hat = inv(X.T.dot(X)).dot(X.T).dot(y)
    y_hat = X.dot(b_hat)
    sse = np.sum((y - y_hat)**2)
    sigma_squared_hat_biased.append(sse / N)
    sigma_squared_hat_unbiased.append(sse / (N-p-1))

In [30]:
np.mean(sigma_squared_hat_biased), np.mean(sigma_squared_hat_unbiased)

(0.6043411635426006, 1.0072352725710012)

In [37]:
sigma_hat = np.sqrt(sigma_squared_hat_unbiased[-1])
V = inv(X.T.dot(X))
v = []
z = []
for j in range(p+1):
    z.append(b_hat[j][0] / sigma_hat / sqrt(V[j,j]))
    v.append(V[j,j])
v = np.array(v)

In [38]:
z

[6.869837989090129,
 8.281062082089798,
 -4.183850330342424,
 0.005054202605464383]

In [39]:
V

array([[ 0.13642588, -0.03195253,  0.15099151, -0.04348441],
       [-0.03195253,  0.40778695, -0.27973261,  0.17571988],
       [ 0.15099151, -0.27973261,  0.90324073,  0.07623619],
       [-0.04348441,  0.17571988,  0.07623619,  0.53766547]])

In [45]:
(b_hat - 1.96*np.sqrt(v)*sigma_hat), (b_hat + 1.96*np.sqrt(v)*sigma_hat)

(array([[ 1.7189479 ,  1.21878002,  0.63949777,  1.04289149],
        [ 4.32624652,  3.82607864,  3.2467964 ,  3.65019011],
        [-4.4551922 , -4.95536008, -5.53464232, -5.13124861],
        [-0.68268862, -1.1828565 , -1.76213875, -1.35874503]]),
 array([[ 3.09135078,  3.59151866,  4.17080091,  3.76740719],
        [ 5.6986494 ,  6.19881728,  6.77809953,  6.37470581],
        [-3.08278932, -2.58262144, -2.00333919, -2.40673291],
        [ 0.68971426,  1.18988214,  1.76916439,  1.36577067]]))